In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys, os, time, gc, random, pickle
from sklearn.neighbors import NearestNeighbors
import requests, shutil
import tensorflow as tf
import keras
from skimage import io
from skimage.transform import resize
from PIL import Image

from keras import backend as K
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

%matplotlib inline

Using TensorFlow backend.


# Load the data set

In [2]:
# Load the data
train = pd.read_csv('./data/all/train.csv')
test = pd.read_csv('./data/all/test.csv')

print('Train:\t\t', train.shape)
print('Test:\t\t', test.shape)
print('Landmarks:\t', len(train['landmark_id'].unique()))

Train:		 (1225029, 3)
Test:		 (117703, 2)
Landmarks:	 14951


In [3]:
train.head()

,id,url,landmark_id
0,cacf8152e2d2ae60,http://static.panoramio.com/photos/original/70...,4676
1,0a58358a2afd3e4e,http://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/...,6651
2,6b2bb500b6a38aa0,http://lh6.ggpht.com/-vKr5G5MEusk/SR6r6SJi6mI/...,11284
3,b399f09dee9c3c67,https://lh3.googleusercontent.com/-LOW2cjAqubA...,8429
4,19ace29d77a5be66,https://lh5.googleusercontent.com/-tnmSXwQcWL8...,6231


# Extract Valid Image IDs

In [4]:
def valid(path):
    """ function to determine whether or not the given image is valid """
    try:
        img = Image.open(path)
        if img.width < 256 or img.height < 256 or img.format != 'JPEG':
            return False
        _ = img.resize((256, 256))
    except:
        return False
    
    return True

In [5]:
if not os.path.exists('./result/train_imgs_id.npy'):
    # Process the training set
    train_imgs_id = []
    train_landmark = []
    t0 = time.time()

    for i in range(len(train)):
        # Helpful information
        if i % 100000 == 0:
            t = time.time() - t0
            print('\nProcess: {:9d}'.format(i), '   Used time: {} s'.format(np.round(t, 0)))
            t0 = time.time()
        if i % 1250 == 0:
            print('=', end='')

        path = './data/all/train_images/' + str(i) + '.jpg'
        if valid(path):
            train_imgs_id.append(i)
            train_landmark.append(train.loc[i, 'landmark_id'])

    np.save('./result/train_imgs_id.npy', train_imgs_id)
    np.save('./result/train_landmark.npy', train_landmark)

In [6]:
if not os.path.exists('./result/test_imgs_id.npy'):
    # Process the test set
    test_imgs_id = []
    t0 = time.time()

    for i in range(len(test)):
        # Helpful information
        if i % 50000 == 0:
            t = time.time() - t0
            print('\nProcess: {:9d}'.format(i), '   Used time: {} s'.format(np.round(t, 0)))
            t0 = time.time()
        if i % 625 == 0:
            print('=', end='')

        path = './data/all/test_images/' + str(i) + '.jpg'
        if valid(path):
            test_imgs_id.append(i)

    np.save('./result/test_imgs_id.npy', test_imgs_id)

# Load fine-tuned InceptionV3 Model

In [7]:
# Load trained model
model = load_model('./models/inception-base-0.1-model.h5')
model.summary()

Instructions for updating:
dim is deprecated, use axis instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 2048)              21802784  
_________________________________________________________________
l2_norm (Lambda)             (None, 2048)              0         
Total params: 21,802,784
Trainable params: 6,073,536
Non-trainable params: 15,729,248
_________________________________________________________________


# Extract training and test image features

In [9]:
# Process all the training images
train_imgs_id = np.load('./result/train_imgs_id.npy')
train_landmark = np.load('./result/train_landmark.npy')

test_imgs_id = np.load('./result/test_imgs_id.npy')
train_features = np.zeros(shape=(len(train_imgs_id), 2048), dtype=np.float32)

print('Train:\t', len(train_imgs_id), len(train_landmark), len(np.unique(train_landmark)), train_features.shape)
print('Test:\t', len(test_imgs_id))

Train:	 1192931 1192931 14943 (1192931, 2048)
Test:	 108383


In [10]:
steps = 20000
t0 = time.time()

for i in range(0, len(train_imgs_id), steps):
    # Helpful information
    if i % steps == 0:
        t = time.time() - t0
        print('\nProcess: {:9d}'.format(i), '   Used time: {} s'.format(np.round(t, 0)))
        t0 = time.time()
        
    tmp_imgs = []
    start = i
    end = min(len(train_imgs_id), i + steps)
    for idx in range(start, end):
        if idx % 250 == 0:
            print('=', end='')
            
        img_id = train_imgs_id[idx]
        path = './data/all/train_images/' + str(img_id) + '.jpg'
        img = load_img(path, target_size=(224, 224))
        img = img_to_array(img)
        tmp_imgs.append(img)
        
    tmp_imgs = np.array(tmp_imgs, dtype=np.float32) / 255.0
    tmp_feature = model.predict(tmp_imgs)
    train_features[start:end, ] = tmp_feature
    _ = gc.collect()

# Save to disk
np.save('./data/all/train_features.npy', train_features)
np.save('./data/all/train_id.npy', train_landmark)


Process:         0    Used time: 0.0 s
Process:     20000    Used time: 545.0 s
Process:     40000    Used time: 544.0 s
Process:     60000    Used time: 546.0 s
Process:     80000    Used time: 579.0 s
Process:    100000    Used time: 573.0 s
Process:    120000    Used time: 572.0 s
Process:    140000    Used time: 596.0 s
Process:    160000    Used time: 599.0 s
Process:    180000    Used time: 588.0 s
Process:    200000    Used time: 594.0 s
Process:    220000    Used time: 607.0 s
Process:    240000    Used time: 602.0 s
Process:    260000    Used time: 599.0 s
Process:    280000    Used time: 605.0 s
Process:    300000    Used time: 601.0 s
Process:    320000    Used time: 601.0 s
Process:    340000    Used time: 605.0 s
Process:    360000    Used time: 600.0 s
Process:    380000    Used time: 606.0 s
Process:    400000    Used time: 615.0 s
Process:    420000    Used time: 608.0 s
Process:    440000    Used time: 613.0 s
Process:    460000    Used time: 614.0 s
Process:    48000

In [12]:
# Process all the test images
test_imgs_id = np.load('./result/test_imgs_id.npy')
test_features = np.zeros(shape=(len(test_imgs_id), 2048), dtype=np.float32)

steps = 20000
t0 = time.time()

for i in range(0, len(test_imgs_id), steps):
    # Helpful information
    if i % steps == 0:
        t = time.time() - t0
        print('\nProcess: {:9d}'.format(i), '   Used time: {} s'.format(np.round(t, 0)))
        t0 = time.time()
        
    tmp_imgs = []
    start = i
    end = min(len(test_imgs_id), i + steps)
    for idx in range(start, end):
        if idx % 250 == 0:
            print('=', end='')
            
        img_id = test_imgs_id[idx]
        path = './data/all/test_images/' + str(img_id) + '.jpg'
        img = load_img(path, target_size=(224, 224))
        img = img_to_array(img)
        tmp_imgs.append(img)
        
    tmp_imgs = np.array(tmp_imgs, dtype=np.float32) / 255.0
    tmp_feature = model.predict(tmp_imgs)
    test_features[start:end, ] = tmp_feature
    _ = gc.collect()

# Save to disk
np.save('./data/all/test_features.npy', test_features)


Process:         0    Used time: 0.0 s
Process:     20000    Used time: 658.0 s
Process:     40000    Used time: 654.0 s
Process:     60000    Used time: 646.0 s
Process:     80000    Used time: 650.0 s
Process:    100000    Used time: 648.0 s